🧠 1. Define the Agents (Using CrewAI)

Each agent will have:

    A specific goal/task

    A unique prompt

    A toolset (e.g., Pandas or custom functions)

    Access to the same dataset

In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

🔹 Agent 1: IntegrityAgent

    Checks for Accuracy, Completeness, Consistency, Timeliness

In [2]:
from crewai import Agent

IntegrityAgent = Agent(
    name="Data Integrity Agent",
    role="Data Quality Inspector",
    goal="Ensure the dataset adheres to accuracy, completeness, consistency, and timeliness standards",
    backstory="You're a data governance expert tasked with validating structural data integrity.",
    tools=[],  # Add pandas tools or custom data tools if needed
    verbose=True
)


 Agent 2: BiasAgent

    Detects Societal Bias in sensitive fields (gender, race, age, etc.)

In [3]:
BiasAgent = Agent(
    name="Societal Bias Agent",
    role="Ethics and Fairness Auditor",
    goal="Analyze the dataset for bias in representation, distribution, and correlations in sensitive features",
    backstory="You're responsible for ensuring fairness in AI data. You assess bias in demographic attributes.",
    tools=[],
    verbose=True
)


🔹 Agent 3: ImbalanceAgent

    Checks for class imbalance in target fields

In [4]:
ImbalanceAgent = Agent(
    name="Data Imbalance Agent",
    role="Imbalance Checker",
    goal="Check for class imbalance in classification labels or target variables, and suggest mitigation strategies",
    backstory="You're a data scientist focused on training balanced ML models.",
    tools=[],
    verbose=True
)


 Agent 4: SecurityAgent

    Looks for PII and security/privacy risks

In [5]:
SecurityAgent = Agent(
    name="Data Security Agent",
    role="Privacy and Security Specialist",
    goal="Inspect data for PII and security vulnerabilities like target leakage or plaintext passwords",
    backstory="You're a data security expert responsible for identifying and remediating privacy risks.",
    tools=[],
    verbose=True
)


In [6]:
from crewai import Task, Crew

task_integrity = Task(
    description="Check the uploaded dataset for data integrity issues and suggest or apply fixes.",
    agent=IntegrityAgent,
    expected_output="A list of integrity issues found in the dataset, with suggested or applied fixes. Include specific examples for accuracy, completeness, consistency, and timeliness."
)


🧩 2. Crew Task & Workflow (CrewAI)

You'll create tasks and define the workflow:

In [7]:
from crewai import Task, Crew

task_integrity = Task(
    description="Check the uploaded dataset for data integrity issues and suggest or apply fixes.",
    agent=IntegrityAgent,
    expected_output="A list of integrity issues found in the dataset, with suggested or applied fixes. Include specific examples for accuracy, completeness, consistency, and timeliness."
)

task_bias = Task(
    description="Analyze the dataset for societal or demographic bias and recommend corrective actions.",
    agent=BiasAgent,
    expected_output="A summary of potential biases related to sensitive attributes and how they might affect downstream models. Include recommendations for corrective actions."
)

task_imbalance = Task(
    description="Evaluate class imbalance in target variables and propose balancing strategies.",
    agent=ImbalanceAgent,
    expected_output="An imbalance ratio report with recommendations for resampling or reweighting methods."
)

task_security = Task(
    description="Scan the dataset for PII and security vulnerabilities, and suggest anonymization or encryption.",
    agent=SecurityAgent,
    expected_output="A list of security/privacy risks found, such as PII exposure or target leakage, and proposed mitigations."
)

crew = Crew(
    agents=[IntegrityAgent, BiasAgent, ImbalanceAgent, SecurityAgent],
    tasks=[task_integrity, task_bias, task_imbalance, task_security],
    verbose=True
)

In [8]:
# Load your dataset (CSV, JSON, etc.)
import pandas as pd
df = pd.read_csv("/content/augmented_credit_card.csv")

# Provide context to all agents
output = crew.kickoff(inputs={"dataset": df.head(20).to_string()})

print(output)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1235f0d1-47ea-430a-9171-3dca80706152                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Quality Inspector                                                                                  │
│                                                                                                                 │
│  Task: Check the uploaded dataset for data integrity issues and suggest or apply fixes.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Quality Inspector                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Accuracy Issues**:                                                                                        │
│     - **Issue Found**: Duplicate records for customer ID 12345, showing different transaction amounts.          │
│       - **Suggested Fix**: Merge records ensuring total transaction amount reflects the sum of both records.    │
│  Update the record to show a single entry for customer ID 12345 with a transaction amount of $5000 instead of   │
│  two separate entries of $3000 and $2000.                                                                       │
│                                                                                                                 │
│  2. **Completeness Issues**:                                                                                    │
│     - **Issue Found**: Missing values in the ‘email’ field for 15% of the records.                              │
│       - **Suggested Fix**: Reach out to customers for missing emails and input the data. For immediate          │
│  solutions, anonymize records with no email to comply with data privacy standards.                              │
│                                                                                                                 │
│  3. **Consistency Issues**:                                                                                     │
│     - **Issue Found**: Inconsistent date formats (MM/DD/YYYY vs. DD/MM/YYYY).                                   │
│       - **Suggested Fix**: Standardize all date fields to the ISO format YYYY-MM-DD across the dataset for      │
│  uniformity. For example, convert “12/31/2021” to “2021-12-31” and “31/12/2021” to “2021-12-31”.                │
│                                                                                                                 │
│  4. **Timeliness Issues**:                                                                                      │
│     - **Issue Found**: Transaction records date back to 2010, raising concerns about relevancy.                 │
│       - **Suggested Fix**: Establish a policy to archive or delete records older than five years unless         │
│  explicitly needed for historical purposes. Archive records prior to 2017 in a separate database.               │
│                                                                                                                 │
│  The applied fixes will enhance the overall quality and usability of the dataset, ensuring adherence to         │
│  accuracy, completeness, consistency, and timeliness standards.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 782cb2be-1869-47c5-8498-b5b8d96d1b29                                                                     │
│  Agent: Data Quality Inspector                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Privacy and Security Specialist                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Privacy and Security Risks Found                                                                           │
│                                                                                                                 │
│  1. **Personally Identifiable Information (PII) Exposure**:                                                     │
│     - **Issue Identified**: The dataset contains PII such as customer emails that are missing in 15% of the     │
│  records. This not only poses a risk of incomplete customer data but also raises compliance concerns.           │
│     - **Proposed Mitigation**: Reach out to customers to complete missing email data. For records that remain   │
│  incomplete, anonymize them and replace sensitive fields with pseudonyms to maintain privacy while retaining    │
│  utility for analysis.                                                                                          │
│                                                                                                                 │
│  2. **Duplicate Records leading to Target Leakage**:                                                            │
│     - **Issue Identified**: Multiple entries for the same customer ID on transaction records can lead to        │
│  double counting and skewed insights.                                                                           │
│     - **Proposed Mitigation**: Merge duplicate records to confirm transaction amounts are accurate. This        │
│  action reduces target leakage, ensuring fairer model outcomes as predictions will be based on a more accurate  │
│  customer representation.                                                                                       │
│                                                                                                                 │
│  3. **Inconsistent Data Formats**:                                                                              │
│     - **Issue Identified**: Inconsistent date formats could lead to erroneous time-based analyses within        │
│  models, which can influence decision-making or marketing strategies targeted at specific demographic groups.   │
│     - **Proposed Mitigation**: Standardize all date fields to the ISO format (YYYY-MM-DD). This ensures         │
│  uniformity across the dataset, enhancing accuracy in time-sensitive analyses while improving the dataset’s     │
│  overall quality.                                                                                               │
│                                                                                                                 │
│  4. **Retention of Outdated Records**:                                                                          │
│     - **Issue Identified**: The presence of transaction records dating back to 2010 raises concerns regarding   │
│  the relevance of historical data.                                                                              │
│     - **Proposed Mitigation**: Implement a clear data retention policy. Archive or delete records older than    │
│  five years, with a provision to retain only those that serve a valid business purpose, ensuring compliance     │
│  with data minimization principles.                                                                             │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 35b4dae4-f825-4cab-9e72-7fa106956448                                                                     │
│  Agent: Privacy and Security Specialist                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1235f0d1-47ea-430a-9171-3dca80706152                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Privacy and Security Risks Found                                                             │
│                                                                                                                 │
│  1. **Personally Identifiable Information (PII) Exposure**:                                                     │
│     - **Issue Identified**: The dataset contains PII such as customer emails that are missing in 15% of the     │
│  records. This not only poses a risk of incomplete customer data but also raises compliance concerns.           │
│     - **Proposed Mitigation**: Reach out to customers to complete missing email data. For records that remain   │
│  incomplete, anonymize them and replace sensitive fields with pseudonyms to maintain privacy while retaining    │
│  utility for analysis.                                                                                          │
│                                                                                                                 │
│  2. **Duplicate Records leading to Target Leakage**:                                                            │
│     - **Issue Identified**: Multiple entries for the same customer ID on transaction records can lead to        │
│  double counting and skewed insights.                                                                           │
│     - **Proposed Mitigation**: Merge duplicate records to confirm transaction amounts are accurate. This        │
│  action reduces target leakage, ensuring fairer model outcomes as predictions will be based on a more accurate  │
│  customer representation.                                                                                       │
│                                                                                                                 │
│  3. **Inconsistent Data Formats**:                                                                              │
│     - **Issue Identified**: Inconsistent date formats could lead to erroneous time-based analyses within        │
│  models, which can influence decision-making or marketing strategies targeted at specific demographic groups.   │
│     - **Proposed Mitigation**: Standardize all date fields to the ISO format (YYYY-MM-DD). This ensures         │
│  uniformity across the dataset, enhancing accuracy in time-sensitive analyses while improving the dataset’s     │
│  overall quality.                                                                                               │
│                                                                                                                 │
│  4. **Retention of Outdated Records**:                                                                          │
│     - **Issue Identified**: The presence of transaction records dating back to 2010 raises concerns regarding   │
│  the relevance of historical data.                                                                              │
│     - **Proposed Mitigation**: Implement a clear data retention policy. Archive or delete records older than    │
│  five years, with a provision to retain only those that serve a valid business purpose, ensuring compliance     │
│  with data minimization principles.                   

### Privacy and Security Risks Found

1. **Personally Identifiable Information (PII) Exposure**:
   - **Issue Identified**: The dataset contains PII such as customer emails that are missing in 15% of the records. This not only poses a risk of incomplete customer data but also raises compliance concerns.
   - **Proposed Mitigation**: Reach out to customers to complete missing email data. For records that remain incomplete, anonymize them and replace sensitive fields with pseudonyms to maintain privacy while retaining utility for analysis.

2. **Duplicate Records leading to Target Leakage**:
   - **Issue Identified**: Multiple entries for the same customer ID on transaction records can lead to double counting and skewed insights.
   - **Proposed Mitigation**: Merge duplicate records to confirm transaction amounts are accurate. This action reduces target leakage, ensuring fairer model outcomes as predictions will be based on a more accurate customer representation.

3. **Inconsistent Data